In [166]:
import numpy as np
import plotly.express as px
import math
import scipy.stats as st
import plotly.figure_factory as ff


$λ$: Taxa de chegada

$μ$: Taxa de atendimento / velocidade do servidor

$ρ$: Percentual de tempo que o sistema permanece ocupado Marcos

$w$: Tempo médio de permanência do cliente no sistema (Fila + Atendimento) OK

$w_q$:Tempo médio de permanência do cliente na fila OK

$L$: Média de clientes no sistema  

$L_q$: Média de clientes na fila

$π_n$: Probabilidade de haver n clientes na fila. 

In [167]:
def simulador(simulation_total_time, mean_arrival_time, mean_departure_time):
    n, simul_time, waiting_line_time, number_of_arrivals, system_spent_time, clients_in_system, clients_in_line, last_event_time = 0, 0, 0, 0, 0, 0, 0, 0
    line_queue, arrivals = [], []

    while(simul_time < simulation_total_time):
        arrival_time = np.random.exponential(scale=1/mean_arrival_time)
        departure_time = np.random.exponential(scale=1/mean_departure_time)
        last_event_time = simul_time
        if n == 0 or arrival_time < departure_time:
            simul_time += arrival_time
            number_of_arrivals +=1
            clients_in_system, clients_in_line = clients_in_system + ((simul_time-last_event_time) * n), clients_in_line + ((simul_time-last_event_time) * len (line_queue))
            n += 1
            arrivals.append(simul_time)
            if (n > 1):
                line_queue.append(simul_time)
        else:
            simul_time += departure_time
            clients_in_system, clients_in_line = clients_in_system + ((simul_time-last_event_time) * n), clients_in_line + ((simul_time-last_event_time) * len (line_queue))
            if (len(line_queue) > 0):
                waiting_line_time += simul_time - line_queue.pop(0)
            n, system_spent_time = n - 1, system_spent_time + (simul_time - arrivals.pop(0)) 

    return n, simul_time, waiting_line_time/number_of_arrivals, system_spent_time/number_of_arrivals, number_of_arrivals, clients_in_system/simul_time, clients_in_line/simul_time

In [168]:
sum([simulador(1000, 1, 2)[2] for _ in range(50)])/50

0.4909078282428663

In [169]:
simulador(1000,1,2)

(1,
 1000.9949190503039,
 0.39945313368688556,
 0.8840995507031156,
 987,
 0.8717389468588534,
 0.39386837579856177)

In [170]:
def simulator_confidence_interval(sample_size, conf_level):
    """
    Simula o processo de fila m/m/1 um intervalo de confiança para tempo médio de espera na fila.

    sample_size: tamanho da amostra para a simulação
    confidence_level: nível de confiança desejado para o intervalo de confiança

    Retorna um intervalo de confiança para o tempo médio de espera na fila.
    """
    results = [simulador(1500, 1, 2)[2] for _ in range(sample_size)]

    # Cálculo do intervalo de confiança
    proportion = np.sum(results) / sample_size
    std_error = math.sqrt(proportion * (1 - proportion) / sample_size)
    margin_error = std_error * st.norm.ppf(conf_level)
    lower_bound = proportion - margin_error #limite superior do IC
    upper_bound = proportion + margin_error  # limite inferior do IC

    return (lower_bound, upper_bound), results

In [171]:
intervalo, data = simulator_confidence_interval(1000, 0.95)
print(intervalo)

(0.4710788629805911, 0.5230928183068381)


In [172]:
px.box(data, orientation='h')

In [173]:
hist_data = [data]
group_labels = ['']

fig = ff.create_distplot(hist_data, group_labels, curve_type='normal', bin_size=0.009)
fig.show()

In [174]:
px.histogram(data, nbins=70)